In [ ]:
    def calculate_angle(self, a, b, c):
        """计算带方向的三维角度（以b为顶点）"""
        # 转换为三维向量
        ba = [a[0] - b[0], a[1] - b[1], a[2] - b[2]]
        bc = [c[0] - b[0], c[1] - b[1], c[2] - b[2]]
        
        # 计算点积和模长
        dot_product = sum(i*j for i,j in zip(ba, bc))
        norm_ba = math.sqrt(sum(i**2 for i in ba))
        norm_bc = math.sqrt(sum(i**2 for i in bc))
        
        # 计算角度（带方向）
        cosine_angle = dot_product / (norm_ba * norm_bc)
        angle = math.degrees(math.acos(max(min(cosine_angle, 1), -1)))
        
        # 通过叉积判断方向
        cross_z = ba[0]*bc[1] - ba[1]*bc[0]
        return angle if cross_z >= 0 else -angle



            # 计算关键角度
            hip_angle = self.calculate_angle(
                keypoints["shoulder"], keypoints["hip"], keypoints["knee"]
            )
            knee_angle = self.calculate_angle(
                keypoints["hip"], keypoints["knee"], keypoints["ankle"]
            )
            
            # 可视化标注
            self.draw_landmarks(frame, keypoints)
            cv2.putText(frame, f"Hip: {hip_angle:.1f}°", (20, 40),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
            cv2.putText(frame, f"Knee: {knee_angle:.1f}°", (20, 80),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

In [ ]:
import sys
import cv2
import math
import time
import pyttsx3
import os
from datetime import datetime
from docx import Document
from docx.shared import Inches
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
import mediapipe as mp

class ThomasTestApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("托马斯紧张度测试系统")
        self.subject_id = ""
        
        # UI控件
        self.input_label = QLabel("受试者编号:")
        self.subject_input = QLineEdit()
        self.start_button = QPushButton("开始测试")
        self.start_button.clicked.connect(self.start_test)
        
        # 视频显示
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        
        layout = QVBoxLayout()
        layout.addWidget(self.input_label)
        layout.addWidget(self.subject_input)
        layout.addWidget(self.start_button)
        layout.addWidget(self.video_label)
        
        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)
        
        # 初始化mediapipe
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
        self.mp_drawing = mp.solutions.drawing_utils
        
        # 测试参数
        self.cap = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.test_stage = 0  # 0=准备 1=测试中 2=完成
        self.symptom_responses = []
        self.record_time = 0
        self.angles_stable = False

    def calculate_angle(self, a, b, c):
        """计算带方向的三维角度（以b为顶点）"""
        # 转换为三维向量
        ba = [a[0] - b[0], a[1] - b[1], a[2] - b[2]]
        bc = [c[0] - b[0], c[1] - b[1], c[2] - b[2]]
        
        # 计算点积和模长
        dot_product = sum(i*j for i,j in zip(ba, bc))
        norm_ba = math.sqrt(sum(i**2 for i in ba))
        norm_bc = math.sqrt(sum(i**2 for i in bc))
        
        # 计算角度（带方向）
        cosine_angle = dot_product / (norm_ba * norm_bc)
        angle = math.degrees(math.acos(max(min(cosine_angle, 1), -1)))
        
        # 通过叉积判断方向
        cross_z = ba[0]*bc[1] - ba[1]*bc[0]
        return angle if cross_z >= 0 else -angle

    def start_test(self):
        """启动测试流程"""
        self.subject_id = self.subject_input.text().strip()
        if not self.subject_id:
            QMessageBox.warning(self, "警告", "请输入受试者编号!")
            return
        
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            QMessageBox.critical(self, "错误", "无法打开摄像头!")
            return
        
        # 显示测试说明
        # instructions = (
        #     "测试说明：\n"
        #     "1. 仰卧于检查床边缘\n"
        #     "2. 双手抱一侧膝至胸前\n"
        #     "3. 被测腿自然下垂\n"
        #     "4. 保持骨盆中立位"
        # )
        # QMessageBox.information(self, "测试说明", instructions)
        
        self.timer.start(30)
        self.start_button.setEnabled(False)
        self.subject_input.setEnabled(False)
        self.test_stage = 1

    def update_frame(self):
        """实时视频处理"""
        ret, frame = self.cap.read()
        if not ret:
            return
        
        frame = cv2.flip(frame, 1)
        results = self.pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            keypoints = {
                "hip": (landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].z),
                "knee": (landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                         landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].z),
                "ankle": (landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
                          landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].z),
                "shoulder": (landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                             landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].z)
            }
            
            # 计算关键角度
            hip_angle = self.calculate_angle(
                keypoints["shoulder"], keypoints["hip"], keypoints["knee"]
            )
            knee_angle = self.calculate_angle(
                keypoints["hip"], keypoints["knee"], keypoints["ankle"]
            )
            
            # 可视化标注
            self.draw_landmarks(frame, keypoints)
            cv2.putText(frame, f"Hip: {hip_angle:.1f}°", (20, 40),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
            cv2.putText(frame, f"Knee: {knee_angle:.1f}°", (20, 80),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

            # 角度稳定性检测
            if self.test_stage == 1:
                if abs(hip_angle - self.last_hip) < 2 and abs(knee_angle - self.last_knee) < 2:
                    if time.time() - self.record_time > 3:  # 稳定3秒
                        self.record_measurement(frame, hip_angle, knee_angle)
                        self.test_stage = 2
                else:
                    self.record_time = time.time()
                
                self.last_hip = hip_angle
                self.last_knee = knee_angle

        # 显示画面
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame.shape
        q_img = QImage(frame.data, w, h, QImage.Format.Format_RGB888)
        self.video_label.setPixmap(QPixmap.fromImage(q_img))

    def draw_landmarks(self, frame, keypoints):
        """绘制关键点和连线"""
        # 绘制连线
        connections = [("shoulder", "hip"), ("hip", "knee"), ("knee", "ankle")]
        for start, end in connections:
            start_pt = (int(keypoints[start][0]*frame.shape[1]), 
                        int(keypoints[start][1]*frame.shape[0]))
            end_pt = (int(keypoints[end][0]*frame.shape[1]), 
                     int(keypoints[end][1]*frame.shape[0]))
            cv2.line(frame, start_pt, end_pt, (0,255,0), 2)
        
        # 绘制关节点
        for name, point in keypoints.items():
            color = (0,0,255) if name == "hip" else (255,0,0)
            center = (int(point[0]*frame.shape[1]), int(point[1]*frame.shape[0]))
            cv2.circle(frame, center, 8, color, -1)

    def record_measurement(self, frame, hip_angle, knee_angle):
        """记录测量结果"""
        # 保存测试快照
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        img_path = os.path.join(os.path.expanduser("~"), "Desktop", 
                               f"Thomas_{self.subject_id}_{timestamp}.jpg")
        cv2.imwrite(img_path, frame)
        
        # 症状收集
        symptoms = []
        symptoms.append(self.ask_symptom("是否感到麻木？"))
        if symptoms[-1] == "否":
            symptoms.append(self.ask_symptom("是否感到疼痛或不适？"))
            symptoms.append(self.ask_symptom("是否感到牵扯感？"))
        
        # 生成诊断报告
        report_content = self.generate_diagnosis(hip_angle, knee_angle, symptoms)
        report_path = os.path.join(os.path.expanduser("~"), "Desktop",
                                  f"Report_{self.subject_id}_{timestamp}.docx")
        self.generate_report(img_path, report_content, report_path)
        
        # 显示结果
        result_msg = f"髋关节角度：{hip_angle:.1f}°\n膝关节角度：{knee_angle:.1f}°\n诊断结果：{report_content}"
        QMessageBox.information(self, "测试完成", result_msg)
        self.reset_test()

    def ask_symptom(self, question):
        """症状提问对话框"""
        msg_box = QMessageBox(self)
        msg_box.setWindowTitle("症状询问")
        msg_box.setText(question)
        yes_btn = msg_box.addButton("是", QMessageBox.YesRole)
        no_btn = msg_box.addButton("否", QMessageBox.NoRole)
        msg_box.exec()
        return "是" if msg_box.clickedButton() == yes_btn else "否"

    def generate_diagnosis(self, hip_angle, knee_angle, symptoms):
        """生成诊断结果（12种组合判断）"""
        # 角度分类
        hip_range = "50~-10" if hip_angle > -10 else "-10~-50"
        knee_range = "180~120" if knee_angle > 120 else "120~80"
        
        # 症状组合
        symptom_key = "".join(["1" if s == "是" else "0" for s in symptoms])
        
        # 诊断规则库
        diagnosis_map = {
            ("50~-10", "180~120", "1"): "髂腰肌紧张伴神经压迫",
            ("50~-10", "180~120", "001"): "髂腰肌轻度紧张",
            ("50~-10", "120~80", "1"): "复合型紧张伴神经症状",
            ("50~-10", "120~80", "001"): "髂腰肌与股直肌复合紧张",
            ("-10~-50", "180~120", "1"): "严重髋屈肌挛缩",
            ("-10~-50", "180~120", "001"): "髋关节囊挛缩",
            ("-10~-50", "120~80", "1"): "全髋屈肌群病变",
            ("-10~-50", "120~80", "001"): "髋膝复合挛缩",
            # 补充其他组合...
        }
        
        return diagnosis_map.get((hip_range, knee_range, symptom_key), 
               "需结合临床进一步检查")

    def generate_report(self, img_path, diagnosis, report_path):
        """生成专业报告文档"""
        doc = Document()
        doc.add_heading('托马斯测试报告', 0)
        
        # 基本信息
        doc.add_paragraph(f"受试者编号：{self.subject_id}")
        doc.add_paragraph(f"测试时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # 诊断结果
        doc.add_heading('诊断结论', level=1)
        doc.add_paragraph(diagnosis)
        
        # 测试图片
        doc.add_heading('体位图示', level=1)
        doc.add_picture(img_path, width=Inches(4))
        
        doc.save(report_path)

    def reset_test(self):
        """重置测试状态"""
        self.test_stage = 0
        self.start_button.setEnabled(True)
        self.subject_input.setEnabled(True)
        self.cap.release()

    def closeEvent(self, event):
        if self.cap and self.cap.isOpened():
            self.cap.release()
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ThomasTestApp()
    window.resize(1024, 768)
    window.show()
    sys.exit(app.exec())

I0000 00:00:1743428053.837811 25732786 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743428053.931294 25733518 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743428053.943364 25733521 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-03-31 21:34:14.021 python[42484:25732786] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-31 21:34:14.021 python[42484:25732786] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-03-31 21:34:15.343 python[42484:25732786] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
2025-03-31 21:34:16.501 python[42484:25732786] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Plea

AttributeError: 'ThomasTestApp' object has no attribute 'last_hip'

AttributeError: 'ThomasTestApp' object has no attribute 'last_hip'

AttributeError: 'ThomasTestApp' object has no attribute 'last_hip'

AttributeError: 'ThomasTestApp' object has no attribute 'last_hip'

...

AttributeError: 'ThomasTestApp' object has no attribute 'last_hip'

AttributeError: 'ThomasTestApp' object has no attribute 'last_hip'

AttributeError: 'ThomasTestApp' object has no attribute 'last_hip'

SystemExit: 0

/Users/zehaokou/miniconda3/envs/rehab/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import cv2
import mediapipe as mp
import math
import pyttsx3
import time
import os
from datetime import datetime
from docx import Document
from docx.shared import Inches

from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QLineEdit, QPushButton,
                             QVBoxLayout, QWidget, QMessageBox)
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QImage, QPixmap

class PoseApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("姿势检测")
        self.subject_id = ""
        
        # UI控件：标签、输入框和按钮
        self.input_label = QLabel("请输入受试者编号:")
        self.subject_input = QLineEdit()
        self.start_button = QPushButton("开始检测")
        self.start_button.clicked.connect(self.start_detection)
        
        # 用于显示摄像头画面的标签
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        
        layout = QVBoxLayout()
        layout.addWidget(self.input_label)
        layout.addWidget(self.subject_input)
        layout.addWidget(self.start_button)
        layout.addWidget(self.video_label)
        
        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)
        
        # 初始化 mediapipe 和 pyttsx3
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.mp_drawing = mp.solutions.drawing_utils
        self.engine = pyttsx3.init()
        
        # 摄像头和计时器
        self.cap = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        
        # 用于逻辑判断的变量
        self.last_time = time.time()
        self.hip_angle_history = []
        self.leg_raised = False

    def speak(self, message):
        # 语音播报并打印消息
        print(message)
        self.engine.say(message)
        self.engine.runAndWait()

    def calculate_angle(self, a, b, c):
        # 计算三点之间的夹角
        ab = [b[0] - a[0], b[1] - a[1]]
        bc = [c[0] - b[0], c[1] - b[1]]
        dot_product = ab[0] * bc[0] + ab[1] * bc[1]
        cross_product = ab[0] * bc[1] - ab[1] * bc[0]
        angle = math.degrees(math.atan2(abs(cross_product), dot_product))
        return angle

    def ask_question(self, question):
        # 使用消息对话框进行简单的“是/否”提问
        msg_box = QMessageBox(self)
        msg_box.setWindowTitle("提问")
        msg_box.setText(question + "\n点击 [是] 或 [否]")
        yes_button = msg_box.addButton("是", QMessageBox.ButtonRole.YesRole)
        no_button = msg_box.addButton("否", QMessageBox.ButtonRole.NoRole)
        msg_box.exec()
        if msg_box.clickedButton() == yes_button:
            return "是"
        else:
            return "否"

    def generate_report_with_image(self, image_path, report_content, report_path):
        doc = Document()
        doc.add_heading('姿势检测报告', 0)
        doc.add_paragraph(f"报告生成时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        # 创建表格
        table = doc.add_table(rows=1, cols=2)
        table.style = 'Table Grid'
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = '症状'
        hdr_cells[1].text = '描述'
        for cell in hdr_cells:
            cell.width = Inches(3)
        symptoms = report_content.split('；')
        for symptom in symptoms:
            row_cells = table.add_row().cells
            symptom_parts = symptom.split('：')
            if len(symptom_parts) == 2:
                row_cells[0].text = symptom_parts[0]
                row_cells[1].text = symptom_parts[1]
        doc.add_paragraph("照片：")
        doc.add_picture(image_path, width=Inches(3))
        doc.save(report_path)

    def start_detection(self):
        # 获取受试者编号，并启动摄像头检测
        self.subject_id = self.subject_input.text().strip()
        if not self.subject_id:
            QMessageBox.warning(self, "警告", "请输入受试者编号!")
            return
        # camera
        # self.cap = cv2.VideoCapture(1)
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            QMessageBox.critical(self, "错误", "无法打开摄像头!")
            return
        self.timer.start(30)  # 每30毫秒更新一次画面
        self.start_button.setEnabled(False)
        self.subject_input.setEnabled(False)

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        # 转为RGB并处理姿势检测
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(frame_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = (landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
            left_hip = (landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y)
            left_knee = (landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y)
            left_ankle = (landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
            right_shoulder = (landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
            right_hip = (landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                          landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value].y)
            right_knee = (landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                           landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value].y)
            
            hip_angle = self.calculate_angle(
                left_shoulder, left_hip, left_knee
            )
            knee_angle = self.calculate_angle(
                left_hip, left_knee, left_ankle
            )
            right_hip_angle = self.calculate_angle(
                right_shoulder, right_hip, right_knee
            )

            hip_angle_complement = hip_angle
            knee_angle_complement = 180 - knee_angle
            right_hip_angle = 180 - right_hip_angle
            

            # 在画面上显示角度信息
            cv2.putText(frame, f"hip: {hip_angle_complement:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"knee: {knee_angle_complement:.2f}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"right hip: {right_hip_angle:.2f}", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.line(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 
                    (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接肩膀和髋
            cv2.line(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 
                    (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接髋和膝
            cv2.line(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 
                    (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接膝和踝


            cv2.circle(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 5, (0, 0, 255), -1)  # 左肩
            cv2.circle(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 5, (0, 255, 255), -1)  # 左髋
            cv2.circle(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 5, (255, 0, 0), -1)  # 左膝
            cv2.circle(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 5, (255, 255, 0), -1)  # 左踝

            # 满足条件时，提示受试者抬腿
            if (-55 < hip_angle_complement < 55) and (80 < knee_angle_complement < 180):
                # self.speak("右腿尽量靠近胸部，左腿保持伸展状态，保持三秒")
                self.leg_raised = True

            if self.leg_raised:

                ##########################################
                # 仅在右腿小于90度时才开始判断，可以修改
                ##########################################

                if right_hip_angle > 90:
                    current_time = time.time()
                    if current_time - self.last_time >= 1:
                        self.last_time = current_time
                        self.speak("右腿未贴近胸部")
                    else:
                        if len(self.hip_angle_history) > 0:
                            hip_angle_diff = abs(hip_angle_complement - self.hip_angle_history[-1])
                        else:
                            hip_angle_diff = float('inf')
                        self.hip_angle_history.append(hip_angle_complement)
                        if len(self.hip_angle_history) > 3:
                            self.hip_angle_history.pop(0)
                        if len(self.hip_angle_history) > 2 and hip_angle_diff < 5:
                            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                            file_name = f"pose_{self.subject_id}_{timestamp}.jpg"
                            file_path = os.path.join(os.path.expanduser("~"), "Desktop", file_name)
                            cv2.putText(frame, f"hip: {hip_angle_complement:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            cv2.putText(frame, f"knee: {knee_angle_complement:.2f}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            cv2.imwrite(file_path, frame)
                            self.speak("照片已拍摄并保存")
                            
                            self.speak("请回答以下问题，若答案为是则点击[是]，否则点击[否]")
                            answer_1 = self.ask_question("是否麻木？")
                            if answer_1 == "是":
                                report_content = (
                                    "神经：坐骨神经卡压、紧张；关节：髋关节囊僵紧；"
                                    "软组织：（梨状肌、股二头肌）水肿、卡压，血液循环差；"
                                    "代偿：骨盆、腰椎代偿；无力：核心、臀肌、腘绳肌无力；"
                                    "调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                )
                            else:
                                answer_2 = self.ask_question("是否疼痛或不适？")
                                if answer_2 == "是":
                                    if hip_angle > 70:
                                        report_content = (
                                            "神经：腰骶丛神经紧张；关节：腰椎小关节僵紧；"
                                            "软组织：腰方肌、腰大肌、臀肌紧张，血液循环差；"
                                            "代偿：骨盆代偿；无力：核心无力；"
                                            "调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                        )
                                    else:
                                        report_content = (
                                            "神经：坐骨神经紧张；关节：骶髂关节、髋关节囊僵紧；"
                                            "软组织：后表链紧张，臀肌、腘绳肌、小腿三头肌紧张，血液循环差；"
                                            "代偿：腰椎代偿；无力：髂腰肌、后表链肌群紧张且无力；"
                                            "调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                        )
                                else:
                                    answer_3 = self.ask_question("是否牵扯？")
                                    if answer_3 == "是":
                                        if hip_angle > 70:
                                            report_content = (
                                                "神经：腰骶丛神经紧张；关节：髋关节囊僵紧；"
                                                "软组织：腘绳肌紧张，血液循环差；代偿：骨盆代偿；"
                                                "无力：核心无力；调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                            )
                                        else:
                                            report_content = (
                                                "神经：坐骨神经紧张；关节：髋关节囊僵紧；"
                                                "软组织：臀肌、腘绳肌、小腿三头肌紧张，血液循环差；"
                                                "代偿：腰椎代偿；无力：髂腰肌、后表链肌群紧张且无力；"
                                                "调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                            )
                                    else:
                                        report_content = "未出现明显症状"
                            report_name = f"report_{self.subject_id}_{timestamp}.docx"
                            report_path = os.path.join(os.path.expanduser("~"), "Desktop", report_name)
                            self.generate_report_with_image(file_path, report_content, report_path)
                            self.speak(f"报告已生成并保存至桌面")
                            self.timer.stop()
                            self.cap.release()
                            cv2.destroyAllWindows()
    
        # 将 frame 转为 QImage 显示到 QLabel 上
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame_rgb.shape
        bytes_per_line = ch * w
        q_img = QImage(frame_rgb.data, w, h, bytes_per_line, QImage.Format.Format_RGB888)
        self.video_label.setPixmap(QPixmap.fromImage(q_img))
        
    def closeEvent(self, event):
        self.timer.stop()
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = PoseApp()
    window.show()
    sys.exit(app.exec())

I0000 00:00:1743431381.413540 25794057 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743431381.513951 25797261 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743431381.528003 25797261 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-03-31 22:29:42.354 python[43752:25794057] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-31 22:29:42.354 python[43752:25794057] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-03-31 22:29:50.355 python[43752:25794057] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
2025-03-31 22:29:51.738 python[43752:25794057] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Plea

右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部
右腿未贴近胸部


SystemExit: 0

/Users/zehaokou/miniconda3/envs/rehab/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
